In [ ]:
#import statements
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

#from keypass import NOAA_api
import pylab as pl
from IPython import display

from functools import partial
import pyproj
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive
import pyspark.sql.functions as f

### $TCO_2$ will be target value of final model. The goal of the ML model will be to fill gaps in the dataset in which the target value is missing (~60% of rows). 

First, the gaps in the input variables for the model were filled. The variables missing >5% of rows [gamma, oxygen, apparent O2 utilization, nitrate, silicate, phosphate, total alkilinity, pH] were filled with individual random forest models. 

Startby loading the filled and filtered dataset.


In [ ]:
df =pd.read_csv('C:\\Users\\ossan\\DATA606\\DATA\\TCO2_filled.csv',index_col=0) #save metrics 
print(len(df))
df['year'] =pd.to_numeric( df['year'])
df['year'] =df['year'].astype(int)
g=df.groupby('year')

In [ ]:
frames = {}
for x,y in g:
    frames[x] = y
frames[list(frames.keys())[-1]]

In [ ]:

var = df.columns 
years = list(frames.keys())
list_updatemenus=[];temp_dict={}
for n, year in enumerate(years):
    visible = [False] * len(years)
    visible[n] = True
    temp_dict = dict(label = str(year),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': 'Year %d' % year}])

    list_updatemenus.append(temp_dict)
    
list_c=[];c_dict={}
for n, year in enumerate(var):
    visible = [False] * len(var)
    visible[n] = True
    c_dict = dict(label = str(var),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': var}])

    list_c.append(c_dict)
    

In [ ]:
#plotly is too slow with complete set 

dftrim = df.iloc[::25,:]
print('len before '+ str(len(df))+' and after trim for plotly '+str(len(dftrim)))

In [ ]:

####the dataset needs ot be cut down for plotly to load 
#how many datapoints per cruise? 
dp = [];count = 0 
for i,g in df.groupby('cruise'):
    if len(g)<50: 
        count = count + 1
    dp.append(len(g))

ax=sns.distplot(dp)
#ax.set_xlim([0,1000])

print('there are '+str(count)+ ' cruises with <100 datapoints out of ' +str(len(df.cruise.unique())) + ' cruises')

In [ ]:
df.columns
col=['Cruise', 'Station',  'Year', 'Month', 'Day', 'Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth',
       'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'Total CO2']

actu=['cruise', 'station', 'year', 'month', 'day', 'bottomdepth', 'maxsampdepth', 'pressure', 'depth',
       'temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']

print(len(actu),len(col))
actu[col.index('Cruise')]

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

In [ ]:
#fig5 = go.Figure()

def plot_compare_lag(var, year):
    col=['Cruise', 'Station',  'Year', 'Month', 'Day', 'Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth',
       'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']

    actu=['cruise', 'station', 'year', 'month', 'day', 'bottomdepth', 'maxsampdepth', 'pressure', 'depth',
       'temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']

    #data = te(s,BalancingAuthority)
    fig5 =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=dftrim["longitude"],
                                  lat=dftrim["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 2, color = dftrim[actu[col.index(var)]], colorscale = 'Inferno',
            cmax = df[actu[col.index(var)]].max(),colorbar_title=var) ),
    layout= dict(title ='Variable: ' +var + ' | Year:'  + str(year.astype(str)[0:4]))  )
    fig5.show()
    return fig5 

interactive(plot_compare_lag, var=col, 
            year=list(np.linspace(1984,2019,(2019-1983))),  )
